In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.compat.v1.keras.layers import GRU


In [4]:
model = keras.models.load_model('./embedding_sentiments')

In [6]:
import pandas as pd
train_set = pd.read_csv("../data/train_gr/train.csv")


In [8]:
import re

def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

x = train_set["user_review"]
x_cleaned = cleanTexts(x)

# Tokenizer 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_cleaned)
x_tokens = tokenizer.texts_to_sequences(x_cleaned)

import json
with open("maxlen.json",mode="r") as F:
    maxlen = json.load(F)


In [11]:
# Pad Tokenised sequences
x_tokens_pad = pad_sequences(x_tokens,maxlen=maxlen["maxlen"])
x_tokens_pad.shape

(17494, 21)

In [14]:
feature_extractor = keras.Model(
    inputs=model.inputs,
    outputs=[layer.output for layer in model.layers],
)
print(x[0])
print(x_cleaned[0]) 
print(x_tokens[0])
print(pad_sequences([x_tokens[0]],maxlen=maxlen["maxlen"]))
feature_extractor(pad_sequences([x_tokens[0]],maxlen=maxlen["maxlen"]).reshape(1,21))[1] # requested embbedding

I'm scared and hearing creepy voices.  So I'll pause for a moment and write a review while I wait for my heart beat to return to atleast somewhat calmer times.  This game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood (but more bubble and 'clean').  Hello 1990's.What charactes there are (that isnot trying to kill me) were likable and a bit odd.  I did do a few noob things though, such as:Oh look a class room full of ghosts from dead children, lets shine my flashlight on them and stand there staring at them..Or, hmm creepy music, I'll turn around and see if I can see what's chasing me.Never before in a game have I been this afraid of finding a locked door.
i m scared and hearing creepy voices   so i ll pause for a moment and write a review while i wait for my heart beat to return to atleast somewhat calmer times   this game is adorable and creepy like my happy tree friends but with the graphics sceme of my childhood  but more bubble and  cl

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.3324464 , -0.966098  ,  0.93294835,  0.3458986 ,  0.1472227 ,
        -0.5522215 ,  0.6037281 , -0.8544234 , -0.38403505,  0.3401982 ]],
      dtype=float32)>

In [15]:
feature_extractor(x_tokens_pad)[1]

<tf.Tensor: shape=(17494, 10), dtype=float32, numpy=
array([[-0.3324464 , -0.9660979 ,  0.93294835, ..., -0.85442346,
        -0.38403505,  0.34019822],
       [ 0.27616048, -0.897205  ,  0.21370235, ..., -0.56942254,
        -0.29262242,  0.6775218 ],
       [-0.61817336, -0.9802528 ,  0.9786215 , ..., -0.9246477 ,
         0.46472743,  0.7306475 ],
       ...,
       [ 0.2377021 ,  0.92442423, -0.6943325 , ...,  0.85971653,
        -0.7256061 , -0.40463376],
       [ 0.519196  ,  0.02329798, -0.29234242, ..., -0.15798683,
        -0.24509442, -0.2130076 ],
       [-0.7072564 , -0.99768126,  0.98328817, ..., -0.9862137 ,
        -0.31048873,  0.7591829 ]], dtype=float32)>

In [24]:
import numpy as np
np.squeeze(feature_extractor(x_tokens_pad)[1])

array([[-0.3324464 , -0.9660979 ,  0.93294835, ..., -0.85442346,
        -0.38403505,  0.34019822],
       [ 0.27616048, -0.897205  ,  0.21370235, ..., -0.56942254,
        -0.29262242,  0.6775218 ],
       [-0.61817336, -0.9802528 ,  0.9786215 , ..., -0.9246477 ,
         0.46472743,  0.7306475 ],
       ...,
       [ 0.2377021 ,  0.92442423, -0.6943325 , ...,  0.85971653,
        -0.7256061 , -0.40463376],
       [ 0.519196  ,  0.02329798, -0.29234242, ..., -0.15798683,
        -0.24509442, -0.2130076 ],
       [-0.7072564 , -0.99768126,  0.98328817, ..., -0.9862137 ,
        -0.31048873,  0.7591829 ]], dtype=float32)

In [28]:
embeds = np.squeeze(feature_extractor(x_tokens_pad)[1])
for i in range(embeds.shape[1]):
    train_set["embedding{}".format(i)] = embeds[:,i]

In [45]:
embedded_mean = train_set.groupby(["title"]).mean()[["embedding0","embedding1","embedding2","embedding3","embedding4","embedding5","embedding6","embedding7","embedding8","embedding9"]]
embedded_sum = train_set.groupby(["title"]).sum()[["embedding0","embedding1","embedding2","embedding3","embedding4","embedding5","embedding6","embedding7","embedding8","embedding9"]]


In [46]:
embedded_mean.to_csv("embeddings_per_game_mean.csv")

In [47]:
embedded_sum.to_csv("embeddings_per_game_sum.csv")